Piotr Janczyk

PJN

# Tagowanie morfosyntaktyczne

In [1]:
import regex
import json
from glob import glob
import requests
import pandas as pd
from collections import Counter
from tqdm import tqdm
from typing import List, Tuple, Dict, NamedTuple
import numpy as np

### Wczytanie tekstów ustaw

In [2]:
def read_file(filename: str) -> str:
    with open(filename, "r") as f:
        return f.read()

filenames = sorted(glob("repo/ustawy-v2/*.txt"))

acts = pd.DataFrame({
  "id": [regex.match(r"repo/ustawy-v2/(\d+_\d+)\.txt", filename).group(1) for filename in filenames],
  "text": [read_file(filename) for filename in filenames]
})

### Użycie KRNNT2 do tagowania i lematyzacji

In [3]:
def tag_text(text: str) -> str:
    response = requests.post("http://localhost:9200", data=text.encode("utf-8"))
    return response.content.decode("utf-8")

In [4]:
acts["tagger_output"] = [tag_text(text) for text in tqdm(acts["text"])]
# acts.to_csv("tagged.csv")
# acts = pd.read_csv("tagged.csv", index_col=0)

In [5]:
acts

,id,text,tagger_output
0,1993_599,"\n\n\n\nDz.U. z 1993 r. Nr 129, poz. 599 \n ...",Dz\tnone\n\tdziennik\tbrev:pun\tdisamb\n.\tnon...
1,1993_602,"\n\n\n\nDz.U. z 1993 r. Nr 129, poz. 602 \n ...",Dz\tnone\n\tdziennik\tbrev:pun\tdisamb\n.\tnon...
2,1993_645,"\n\n\n\nDz.U. z 1993 r. Nr 134, poz. 645\n ...",Dz\tnone\n\tdziennik\tbrev:pun\tdisamb\n.\tnon...
3,1993_646,"\n\n\n\nDz.U. z 1993 r. Nr 134, poz. 646\n ...",Dz\tnone\n\tdziennik\tbrev:pun\tdisamb\n.\tnon...
4,1994_150,"\n\n\n\nDz.U. z 1994 r. Nr 40, poz. 150\n ...",Dz\tnone\n\tdziennik\tbrev:pun\tdisamb\n.\tnon...
...,...,...,...
1174,2004_96,\n\n\nTekst ustawy\nprzyjęty przez Senat bez p...,Tekst\tnone\n\ttekst\tsubst:sg:nom:m3\tdisamb\...
1175,2004_962,\n\n\nTekst ustawy\nprzyjęty przez Senat bez p...,Tekst\tnone\n\ttekst\tsubst:sg:nom:m3\tdisamb\...
1176,2004_963,"\n\n\n\nDz.U. z 2004 r. Nr 97, poz. 963\n ...",Dz\tnone\n\tdziennik\tbrev:pun\tdisamb\n.\tnon...
1177,2004_964,\n\n\nTekst ustawy przyjęty przez Senat bez po...,Tekst\tnone\n\ttekst\tsubst:sg:nom:m3\tdisamb\...


### Zliczenie bigramów

In [6]:
tagger_output: str = "\n".join(acts["tagger_output"])

In [7]:
class Token(NamedTuple):
    word: str
    category: str

    def __repr__(self):
        return self.word + ":" + self.category

def extract_tokens(tagger_output: str) -> List[Token]:
    lines = tagger_output.split("\n")
    tokens = [line.strip().split("\t") for line in lines if line.startswith("\t")]
    tokens = [Token(word=token[0].lower(), category=token[1].split(":")[0]) for token in tokens]
    return tokens

In [8]:
tokens: List[Token] = extract_tokens(tagger_output)

In [9]:
Bigram = Tuple[Token, Token]

def compute_bigram_statistics(tokens: List[Token]) -> pd.DataFrame:
    bigrams = list(zip(tokens[:-1], tokens[1:]))
    bigrams = Counter(bigrams)
    bigrams = list(sorted(bigrams.items(), key=lambda item: -item[1]))
    return pd.DataFrame.from_records(bigrams, columns=["bigram", "count"])

In [10]:
bigrams = compute_bigram_statistics(tokens)

In [11]:
bigrams

,bigram,count
0,"((artykuł, brev), (., interp))",83738
1,"((ustęp, brev), (., interp))",53315
2,"((pozycja, brev), (., interp))",45197
3,"((,, interp), (pozycja, brev))",43168
4,"((., interp), (1, adj))",39986
...,...,...
479959,"((ktry, subst), (nie, qub))",1
479960,"((23, adj), (zdanie, subst))",1
479961,"((nabycie, subst), (środkw, subst))",1
479962,"((środkw, subst), (niebezpieczny, adj))",1


### Odrzucenie bigramów zawierających znaki niebędące literami

In [12]:
def is_letter_only_bigram(bigram: Bigram) -> bool:
    letters_only = r"\p{L}+"
    return bool(regex.fullmatch(letters_only, bigram[0].word)) and bool(regex.fullmatch(letters_only, bigram[1].word))

In [13]:
letter_bigrams = bigrams[bigrams["bigram"].apply(is_letter_only_bigram)].reset_index(drop=True)

In [14]:
letter_bigrams

,bigram,count
0,"((w, prep), (artykuł, brev))",32035
1,"((o, prep), (który, adj))",28656
2,"((który, adj), (mowa, subst))",28538
3,"((mowa, subst), (w, prep))",28473
4,"((w, prep), (ustęp, brev))",23536
...,...,...
363862,"((dostawa, subst), (produktw, subst))",1
363863,"((produktw, subst), (w, prep))",1
363864,"((ktry, subst), (nie, qub))",1
363865,"((nabycie, subst), (środkw, subst))",1


### Policzenie LLR

In [15]:
def entropy(x: np.ndarray) -> float:
    x = x.flatten()
    x = x[x != 0]
    probabilities = x / np.sum(x)
    return np.sum(probabilities * np.log(probabilities))

def llr(all: int, a: int, b: int, a_and_b: int) -> float:
    k = np.array([
        [a_and_b, a - a_and_b],
        [b - a_and_b, all - a - b + a_and_b]
    ])
    return 2 * np.sum(k) * (entropy(k) - entropy(np.sum(k, axis=0)) - entropy(np.sum(k, axis=1)))

In [16]:
def compute_bigrams_llr(bigrams: pd.DataFrame) -> List[float]:
    all_count = bigrams["count"].sum()
    df = pd.DataFrame({
        "token1": [bigram[0] for bigram in bigrams["bigram"]],
        "token2": [bigram[1] for bigram in bigrams["bigram"]],
        "count": bigrams["count"],
    })
    count_by_token1 = df.groupby("token1")["count"].sum()
    count_by_token2 = df.groupby("token2")["count"].sum()
    return [
        llr(
            all=all_count,
            a=count_by_token1[bigram[0]],
            b=count_by_token2[bigram[1]],
            a_and_b=count
        )
        for bigram, count in tqdm(zip(bigrams["bigram"], bigrams["count"]))
    ]

In [17]:
letter_bigrams["llr"] = compute_bigrams_llr(letter_bigrams)
letter_bigrams = letter_bigrams.sort_values(by="llr", ascending=False)

363867it [02:42, 2240.76it/s]


In [18]:
letter_bigrams

,bigram,count,llr
2,"((który, adj), (mowa, subst))",28538,2.480508e+05
1,"((o, prep), (który, adj))",28656,1.904906e+05
3,"((mowa, subst), (w, prep))",28473,1.770681e+05
0,"((w, prep), (artykuł, brev))",32035,1.138613e+05
6,"((otrzymywać, fin), (brzmienie, subst))",10535,1.107467e+05
...,...,...,...
309367,"((gmina, subst), (teren, subst))",1,1.606023e-08
274896,"((rachunek, subst), (zgodnie, adv))",1,1.283288e-08
52220,"((i, conj), (wystąpienie, subst))",6,8.262006e-09
112408,"((i, conj), (obniżenie, subst))",3,4.743003e-09


### Podzielenie bigramów wg kategorii morfosyntaktycznych

In [19]:
letter_bigrams["partition"] = letter_bigrams["bigram"].apply(
    lambda bigram: (bigram[0].category, bigram[1].category)
)

In [20]:
letter_bigrams

,bigram,count,llr,partition
2,"((który, adj), (mowa, subst))",28538,2.480508e+05,"(adj, subst)"
1,"((o, prep), (który, adj))",28656,1.904906e+05,"(prep, adj)"
3,"((mowa, subst), (w, prep))",28473,1.770681e+05,"(subst, prep)"
0,"((w, prep), (artykuł, brev))",32035,1.138613e+05,"(prep, brev)"
6,"((otrzymywać, fin), (brzmienie, subst))",10535,1.107467e+05,"(fin, subst)"
...,...,...,...,...
309367,"((gmina, subst), (teren, subst))",1,1.606023e-08,"(subst, subst)"
274896,"((rachunek, subst), (zgodnie, adv))",1,1.283288e-08,"(subst, adv)"
52220,"((i, conj), (wystąpienie, subst))",6,8.262006e-09,"(conj, subst)"
112408,"((i, conj), (obniżenie, subst))",3,4.743003e-09,"(conj, subst)"


In [27]:
largest_partitions = letter_bigrams.groupby("partition")["count"].sum().sort_values(ascending=False).head(10)

In [28]:
largest_partitions

partition
(prep, subst)     326526
(subst, subst)    292525
(subst, adj)      273416
(adj, subst)      187562
(subst, prep)     173132
(subst, conj)      84881
(conj, subst)      83923
(prep, adj)        79171
(ger, subst)       76137
(prep, brev)       66877
Name: count, dtype: int64

### Dla każdego z 10 największych podziałów wypisanie 5 bigramów z największą miarą LLR

In [35]:
for partition in largest_partitions.index:
    print(f"Partition: {partition[0]} {partition[1]}")
    for bigram in letter_bigrams[letter_bigrams["partition"] == partition]["bigram"].head(5):
        count = int(letter_bigrams[letter_bigrams["bigram"] == bigram]["count"])
        llr = round(float(letter_bigrams[letter_bigrams["bigram"] == bigram]["llr"]))
        print(f" - {bigram[0].word} {bigram[1].word}  (N: {count}, LLR: {llr})")
    print()

Partition: prep subst
 - z dzień  (N: 11360, LLR: 53446)
 - na podstawa  (N: 6681, LLR: 47053)
 - do sprawa  (N: 8718, LLR: 46295)
 - w droga  (N: 7128, LLR: 32014)
 - od dzień  (N: 5324, LLR: 31546)

Partition: subst subst
 - droga rozporządzenie  (N: 4748, LLR: 53908)
 - skarb państwo  (N: 1821, LLR: 22069)
 - rada minister  (N: 2268, LLR: 18324)
 - terytorium rzeczpospolita  (N: 1224, LLR: 14072)
 - ochrona środowisko  (N: 1572, LLR: 14017)

Partition: subst adj
 - minister właściwy  (N: 7933, LLR: 70846)
 - rzeczpospolita polski  (N: 3579, LLR: 41566)
 - jednostka organizacyjny  (N: 2252, LLR: 24502)
 - samorząd terytorialny  (N: 1675, LLR: 23387)
 - produkt leczniczy  (N: 1738, LLR: 21898)

Partition: adj subst
 - który mowa  (N: 28538, LLR: 248051)
 - niniejszy ustawa  (N: 2364, LLR: 21433)
 - następujący zmiana  (N: 1624, LLR: 18150)
 - odrębny przepis  (N: 1449, LLR: 12945)
 - walny zgromadzenie  (N: 598, LLR: 9630)

Partition: subst prep
 - mowa w  (N: 28473, LLR: 177068)
 - u

### Wnioski

1. What types of bigrams have been found?

  - Nazwy własne ("Skarb Państwa", "Rada Ministrów", "Rzeczpospolita Polska")
  - Kolokacje czasownik+rzeczownik ("zasięgnąć opinii", "pozbawić wolnosci", "wszcząć postępowanie")
  - Kolokacje z przyimkiem ("z dnia", "na podstawie", "wniosek o")
  - Bigramy zawierające stop words (najczęściej spójniki) ("przecinek i", "wolność albo", "i nazwisko") 


2. Which of the category-pairs indicate valuable multiword expressions? Do they have anything in common?

  Zazwyczaj jednostki wielowyrazowe zawierają rzeczownik lub czasownik.
  Na przykład "subst subst" oraz "subst adj" to często nazwy własne.


3. Which signal: LLR score or syntactic category is more useful for determining genuine multiword expressions?

  Wiele często występujących bigramów składa się ze spójników i przyimków i nie tworzą one jednostek wielowyrazowych. Filtrując po kategoriach syntaktycznych możemy odrzucić takie przypadki.
  Z drugiej strony, bigramy które mają niską miarę LLR (zazwyczaj jest to powiązane z niską liczbą wystąpień), mogą tworzyć przypadkowe pary wyrazów, nie będące jednostkami wielowyrazowymi, ani nawet kolokacjami.
Najlepsze wyniki daje uwzględnienie obu czynników.


4. Can you describe a different use-case where the morphosyntactic category is useful for resolving a real-world problem?

    Przy wyszukiwaniu informacji w tekscie niektóre części mowy (np. rzeczowniki i czasowniki) mają większe znaczenie niż inne (np. spójniki i przyimiki). Odfiltrowanie mniej istotnych kategorii morfosyntaktycznych mogłoby działać podobnie do odfiltrowywania stop words.




